# Deploy DL Model on Web & Mobile Using Pyotrch
---
## Milestone 1: Build an image classifier
## Partial Solutions Notebook
---
#### Date updated:  12-September-2021
#### Author:  Nidhin Pattaniyil & Reshama Shaikh


In [1]:
%matplotlib widget

In [2]:
!ls

00_dataset.ipynb	     __pycache__
01_train.py		     ant_transfer_learning_tutorial.ipynb
01_train_out.txt	     artifacts
01_training.ipynb	     data
01_training__output.ipynb    milestone1_partial_solution_3classes.ipynb
01_training_raw_food.ipynb   requirements.txt
02_torch_optimization.ipynb  utility.py
Untitled.ipynb		     wandb
Untitled1.ipynb


## Imports

In [8]:
# from datetime import date
# from datetime import datetime
import datetime
import plotly.figure_factory as ff
import wandb
from matplotlib import pyplot as plt
import sklearn
import pytorch_lightning as pl
import flash
import flash.image
from flash.core.data.utils import download_data
# from flash.image import ImageClassificationData, ImageClassifier
import os
import glob
import pandas as pd
import sklearn.model_selection
#from sklearn.model_selection import train_test_split 
#from flash.image.classification.adapters import TRAINING_STRATEGIES
# from flash.image.classification.backbones import IMAGE_CLASSIFIER_BACKBONES
# from flash.core.classification import Probabilities
import numpy as np
import torch
from torchvision import transforms as T
import flash
from flash.core.data.data_source import DefaultDataKeys
from flash.core.data.transforms import ApplyToKeys, merge_transforms
# from flash.image import ImageClassificationData, ImageClassifier
# from flash.image.classification.transforms import default_transforms
import flash.core.integrations.fiftyone
import flash.core.classification 
import fiftyone as fo
import warnings
import itertools
import timeit
import json
import torchinfo
import utility

In [ ]:
pl.seed_everything(99)

In [ ]:
flash.__version__

## Parameters


In [ ]:
model_name = 'resnet34'

## Helper code

In [47]:
# def create_fiftyone_dataset(df:pd.DataFrame, name="test_df"):
#     samples = []
    
#     if fo.dataset_exists(name):
#         fo.delete_dataset(name)
#     for record in df.to_dict(orient='records'):
#         sample = fo.Sample(
#                 filepath=record['file_path'],
#                 ground_truth=fo.Classification(label=record['label']),
#             )
        
#         sample['ground_truth'] = fo.Classification(label=record['label'])
#         samples.append(sample)
#     dataset = fo.Dataset(name)
#     dataset.add_samples(samples)
#     return dataset

In [ ]:
!ls

In [ ]:


current_date = datetime.date.today()
print("Today's date:", current_date)

In [ ]:
now1 = datetime.datetime.now()

start_time = now1.strftime("%H:%M:%S")
print("Start Time =", start_time)

In [ ]:
#!pip install lightning-flash==0.5.0

In [ ]:
# get the backbones available for ImageClassifier
backbones = flash.image.ImageClassifier.available_backbones()

# print the backbones
#print(backbones)

In [ ]:
warnings.filterwarnings("ignore", message="Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.")

In [ ]:

# https://lp-prod-resources.s3-us-west-2.amazonaws.com/other/Deploying+a+Deep+Learning+Model+on+Web+and+Mobile+Applications+Using+TensorFlow/Food+101+-+Data+Subset.zip

In [ ]:
#download_data("https://pl-flash-data.s3.amazonaws.com/hymenoptera_data.zip", 'data/')


## Prepare dataset

In [ ]:
data_dir = "data/food-101-subset/images/"

In [ ]:
# files = glob.glob(f"{data_dir}/*/*.jpg")
# df = pd.DataFrame({'file_path':files})
# df['label_name'] = df['file_path'].apply(lambda x:x.split("/")[-2])
# df.head()

In [ ]:
df = pd.read_csv("data/food-101/meta/train.txt",header=None,names=['file_path'])
df['label_name'] = df['file_path'].apply(lambda x:x.split("/")[-2])

df['file_path'] = "data/food-101/images/" + df["file_path"]+".jpg"




In [ ]:
df.head()

In [ ]:
label_encoder = sklearn.preprocessing.LabelEncoder()


In [ ]:
label_encoder.fit(df['label_name'])


In [ ]:
df['label'] = label_encoder.transform(df['label_name'])


In [ ]:
df_eval = pd.read_csv("data/food-101/meta/test.txt",header=None,names=['file_path'])
df_eval['label_name'] = df_eval['file_path'].apply(lambda x:x.split("/")[-2])
df_eval['file_path'] = "data/food-101/images/" + df_eval["file_path"]+".jpg"
df_eval['label'] = label_encoder.transform(df_eval['label_name'])


In [ ]:
#!ls data/food-101/images

In [ ]:
df_eval.head()

In [ ]:
# split the dataset; 85% for training and 15 % for testing
df_train, df_test = sklearn.model_selection.train_test_split(df, test_size=0.15, stratify=df['label'],shuffle = True) 

# split the training dataset: 80% for actual training and 20% for validation
df_train, df_val = sklearn.model_selection.train_test_split(df_train, test_size=0.20, random_state=1 , stratify=df_train['label'])

In [ ]:
print ({"train": len(df_train), "test":len(df_test), "val":len(df_val) ,"eval": len(df_eval)  })

In [ ]:
default_transforms = flash.image.classification.transforms.default_transforms((224, 224))
default_transforms

In [ ]:
post_tensor_transform = ApplyToKeys(
    DefaultDataKeys.INPUT,
    T.Compose([T.RandomHorizontalFlip(), T.ColorJitter(), T.RandomAutocontrast(), T.RandomPerspective()]),
)

new_transforms = merge_transforms(default_transforms, {"post_tensor_transform": post_tensor_transform})

In [ ]:
# datamodule = ImageClassificationData.from_data_frame(
#     "file_path",
#     "label",
#     train_data_frame=df_train,
#     val_data_frame=df_val,
#     test_data_frame=df_test,
#     train_transform=new_transforms
# )

In [ ]:
fo.delete_non_persistent_datasets()

In [49]:
# train_dataset = create_fiftyone_dataset (df_train,"train")
# val_dataset = create_fiftyone_dataset (df_val,"validation") 
# test_dataset = create_fiftyone_dataset (df_test,"test")
# eval_dataset = create_fiftyone_dataset (df_eval,"eval")




 100% |█████████████████████| 0/0 [450.6us elapsed, ? remaining, ? samples/s] 
 100% |█████████████████████| 0/0 [463.0us elapsed, ? remaining, ? samples/s] 
 100% |█████████████████████| 0/0 [373.8us elapsed, ? remaining, ? samples/s] 
 100% |█████████████████████| 0/0 [364.7us elapsed, ? remaining, ? samples/s] 


In [56]:
# ??fo.Dataset

In [40]:
# datamodule = flash.image.ImageClassificationData.from_fiftyone(
#     train_dataset=train_dataset,
#     val_dataset=val_dataset,
#     test_dataset=test_dataset,
#     train_transform=new_transforms,
#      batch_size=64,
#      num_workers=16
# )

ValueError: Dataset has no field 'ground_truth'

In [ ]:
datamodule

In [ ]:
train_dataset

In [ ]:
datamodule.num_classes


In [22]:
def prepare_dataset(data_dir:str)->flash.core.data.data_module.DataModule :
    
    df_train = pd.read_parquet(f"{data_dir}/df_train.parquet" )
    df_test = pd.read_parquet(f"{data_dir}/df_test.parquet")
    df_val = pd.read_parquet(f"{data_dir}/df_val.parquet")
    df_eval = pd.read_parquet(f"{data_dir}/df_eval.parquet")
    
    
#     train_dataset = utility.create_fiftyone_dataset (df_train,"train")
#     val_dataset = utility.create_fiftyone_dataset (df_val,"validation") 
#     test_dataset = utility.create_fiftyone_dataset (df_test,"test")

    train_dataset = create_fiftyone_dataset (df_train,"train")
    val_dataset = create_fiftyone_dataset (df_val,"validation") 
    test_dataset = create_fiftyone_dataset (df_test,"test")
    
    new_transforms = utility.image_transforms()
    
    datamodule = flash.image.ImageClassificationData.from_fiftyone(
        train_dataset=train_dataset,
#         val_dataset=val_dataset,
#         test_dataset=test_dataset,
        # train_transform=new_transforms,
    )
    
    return datamodule , df_eval
    
#    return None, None

In [23]:
datamodule , df_eval = prepare_dataset("artifacts/data")

 100% |█████████████████████| 0/0 [403.7us elapsed, ? remaining, ? samples/s] 
 100% |█████████████████████| 0/0 [582.1us elapsed, ? remaining, ? samples/s] 
 100% |█████████████████████| 0/0 [665.9us elapsed, ? remaining, ? samples/s] 


ValueError: Dataset has no field 'ground_truth'

## Model

In [ ]:
model_metrics = {}

In [ ]:
sample_image = "data/food-101-subset/images/apple_pie/1005649.jpg"
strategy="dp"


In [ ]:
sample_input_size = (1, 3, 224, 224)


In [ ]:
def time_inference(model,device='cpu'):
    model = model.eval()

    with torch.no_grad():
        model.to(device)
        predictions = model.predict(sample_image)


In [ ]:
def train_model(model_name:str, epochs = 2, unfreeze_epoch = 5 ,num_iterations=10, df_eval=df_eval  ):
    model = flash.image.ImageClassifier(num_classes=datamodule.num_classes,backbone=model_name)
    print (f"Model Spec: {model_name}")
    print(model)
    
    
    
    print ("Model Summary")
    
    model_summary = torchinfo.summary(model, input_size=sample_input_size, verbose=0)
    print (model_summary)
    
    
    
            
    
    trainer = flash.Trainer(max_epochs=epochs, gpus=torch.cuda.device_count() , strategy=strategy)

    
    
    trainer.finetune(model, datamodule=datamodule
                 , strategy=flash.core.finetuning.FreezeUnfreeze(unfreeze_epoch=unfreeze_epoch) )
    
    
    metrics_test = trainer.test(model, datamodule=datamodule)
    print (metrics_test)

    
    metrics_val =trainer.validate(model, datamodule=datamodule)
    print(metrics_val)
    
    artifact_model_path = f"artifacts/model_{model_name}.pt"
    trainer.save_checkpoint(artifact_model_path)

    model = flash.image.ImageClassifier.load_from_checkpoint(artifact_model_path)
    
    time_gpu = timeit.timeit(lambda: time_inference (model,device='cuda:0') ,number=num_iterations   ) 
    time_cpu = timeit.timeit(lambda: time_inference (model,device='cpu') ,number=num_iterations   ) 
    
    
    # eval dataset accuracy
    eval_dataset = create_fiftyone_dataset (df_eval,"eval")
    datamodule_eval = flash.image.ImageClassificationData.from_fiftyone(predict_dataset=eval_dataset)
    predictions = trainer.predict(model, datamodule=datamodule_eval) # datamodule_predict
    predictions = list(itertools.chain.from_iterable(predictions))  # flatten batches
    y_proba = predictions
    y_pred = np.argmax(y_proba, axis=1)
    y_true = df_test['label']

    eval_accuracy = sklearn.metrics.accuracy_score(y_true, y_pred)
    
    
    
    res = {**metrics_test[0], **metrics_val[0]}
    res['time_cpu'] = time_cpu
    res['time_gpu'] = time_gpu
    res['eval_accuracy'] = eval_accuracy
    
    res['params_total'] = model_summary.total_params
    res['params_trainable'] = model_summary.trainable_params
    


    model_metrics[model_name] = res
    
    
    

    return model
    
    

In [ ]:
max_epochs = 30
unfreeze_epoch = 10

In [ ]:
trainer = flash.Trainer(max_epochs=max_epochs, gpus=torch.cuda.device_count(), strategy=strategy  )

In [ ]:
model_name = 'vgg19'
model = train_model(model_name=model_name, epochs = max_epochs, unfreeze_epoch = unfreeze_epoch)


In [ ]:
train_model(model_name='resnet50', epochs = max_epochs, unfreeze_epoch = unfreeze_epoch)

In [ ]:
train_model(model_name='mobilenet_v2', epochs = max_epochs, unfreeze_epoch = unfreeze_epoch)

In [ ]:
train_model(model_name='mobilenetv3_small_100', epochs = max_epochs, unfreeze_epoch = unfreeze_epoch)

In [ ]:
model_metrics

In [ ]:
with open("artifacts/model_metrics.json") as f:
    payload = json.dumps(model_metrics)
    f.write(payload)


In [ ]:
df_train.to_parquet("artifacts/data/df_train.parquet")
df_test.to_parquet("artifacts/data/df_test.parquet")
df_val.to_parquet("artifacts/data/df_val.parquet")
df_eval.to_parquet("artifacts/data/adf_eval.parquet")

## Best Model

In [ ]:
model_name = "mobilenet_v2"
artifact_model_path = f"artifacts/model_{model_name}.pt"

In [ ]:
model = flash.image.ImageClassifier.load_from_checkpoint(artifact_model_path)
model

In [ ]:
model_summary = torchinfo.summary(model, input_size=sample_input_size, verbose=0)
model_summary

## Predict on Single

In [ ]:
model.serializer = flash.core.classification.Probabilities()

In [ ]:
predictions = model.predict("data/food-101-subset/images/apple_pie/1005649.jpg")
predictions
#label_encoder.inverse_transform(predictions)

In [ ]:
predictions[0]

In [ ]:
sorted ( zip (label_encoder.classes_, predictions[0]), key= lambda x:x[1] ,reverse =True)

In [ ]:
# datamodule_predict = ImageClassificationData.from_data_frame(
#     "file_path",
#     "label",
#     test_data_frame=df_test,
#     predict_transform=default_transforms
# )
# datamodule_predict

## Predict on dataset

In [ ]:
#?flash.core.classification.PredsClassificationSerializer

In [ ]:
# model.serializer = flash.core.classification.PredsClassificationSerializer()
# fo_predict = create_fiftyone_dataset(df_test,"predict")
# datamodule_predict = ImageClassificationData.from_fiftyone(fo_predict)


In [ ]:
model.serializer = flash.core.classification.FiftyOneLabels(return_filepath=False, labels=label_encoder.classes_)  # output FiftyOne format
fo_predict = create_fiftyone_dataset(df_eval,"predict")
datamodule_predict = flash.image.ImageClassificationData.from_fiftyone(predict_dataset=fo_predict)


In [ ]:
predictions = trainer.predict(model, datamodule=datamodule_predict) # datamodule_predict
predictions = list(itertools.chain.from_iterable(predictions))  # flatten batches

fo_predict.set_values("predictions", predictions)

In [ ]:
predictions[0]

In [ ]:
results = fo_predict.evaluate_classifications("predictions", gt_field="ground_truth", eval_key="eval")
results.print_report()

In [ ]:
plot = results.plot_confusion_matrix(classes=label_encoder.classes_)
plot.show()

In [ ]:
# 8 Visualize results in the App
session = fo.launch_app(fo_predict, auto=False)



In [ ]:
session.open_tab()


In [ ]:
!ip a

In [ ]:
# Plot confusion matrix
plot = results.plot_confusion_matrix(classes=label_encoder.classes_)
plot.show(height=600)



In [ ]:
# Connect to session
#session.plots.attach(plot)

In [ ]:
# # Optional: block execution until App is closed
# session.wait()

## Prediction using datamodule

In [ ]:
model.serializer = flash.core.classification.Probabilities()

predictions = trainer.predict(model,datamodule=datamodule_predict)
predictions = list(itertools.chain.from_iterable(predictions))  # flatten batches


In [ ]:
# y_proba = model.predict(list ( df_test['file_path'] ) )
# y_pred = np.argmax(y_proba, axis=1)
# y_true = df_test['label']

In [ ]:
y_proba = predictions
y_pred = np.argmax(y_proba, axis=1)
y_true = df_test['label']

In [ ]:
# https://stackoverflow.com/questions/60860121/plotly-how-to-make-an-annotated-confusion-matrix-using-a-heatmap
fig, ax = plt.subplots(figsize=(15,15))
cm_labels = label_encoder.classes_
disp = sklearn.metrics.ConfusionMatrixDisplay.from_predictions(
    y_true=y_true, y_pred = y_pred ,
    #normalize = 'true',
    display_labels=cm_labels,
    xticks_rotation='vertical',
    ax=ax
)
disp

## Model inference

In [ ]:
sample_image = "data/food-101-subset/images/apple_pie/1005649.jpg"

In [ ]:
model.eval()

In [ ]:
%%timeit -r 9 -n 5
time_inference (model,device='cuda:0')


In [ ]:
%%timeit -r 9 -n 5
time_inference (model,device='cpu')


# Get Model Size

In [ ]:
! du -h artifacts

In [ ]:
!ls artifacts/ -lah

# Compare Models

- Dataset:  Food
- Classes: 3
- Total images: 3000
- Batch size: 32

| Model  | Trainable Params| Non-trainable Params | Time (hh:mm:ss) [a]| Accuracy | Epochs | Model Size [b]
|---|---:|---:|---:|---:| ---:|---:|
| VGG19| 528,387   | 20,024,384 | 00:00:48 | 96.8% | 3 | 83M
| ResNet50   | xxx   | xxx |  xx:xx:xx | xx.x% | 3 | xxM
| MobileNetV2  | xxx  | xxx |   xx:xx:xx | xx.x% | 3 | xxM
| ResNet50_ft [c]  | xxx  | xxx | xx:xx:xx   | xx.x% | xx | xxM

NOTES:  
- [a] If wall clock time < CPU time, then you're executing a program in parallel.
- [b] model size is size of output file
- [c] ft = fine-tuned; time 2min 31s; 3min 59s; epochs 9+5


# Download Assets
download model and classes.json

In [ ]:
#!zip -r artifacts/artifacts.zip {PROJECT_NAME}/artifacts/

In [ ]:
# from google.colab import files
# files.download(str(str(PROJECT_NAME)+"/artifacts/artifacts.zip"))